<a href="https://colab.research.google.com/github/williamedwardhahn/OISC/blob/main/Hahn_MPCR_OISC_May_2023.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# MPCR OISC

In [ ]:
# Thing <- Name of the thing <- Name of the Name of the thing
# 0 Location of IP
# M[IP] Value of IP
# a  Location of Location of Value to be Copied
# b  Location of Location of Value to be Pasted
# i  Location of Value to be Copied
# j  Location of Value to be Pasted
# x  Value to be Copied
# a,b  = M[IP]-2,M[IP]-1 
# i,j  = M[a]   ,M[b]
# i,j  = M[M[IP]-2]   ,M[M[IP]-1]
# x    = M[i]
# y    = M[j]
# y    = x
# M[j] = M[i]
# M[M[M[IP]-1]] = M[M[M[IP]-2]]

In [ ]:
import numpy as np

In [ ]:
%%writefile code.ai
IP
A
B
C
X
Y
Z
Add
Sub
Mult
Div
Equal
Greater
Lesser
P
L
S
W
Halt 0,L L,0
Push A,S W,0
Peek S,A A,S W,0
Pop S,A W,0
Drop S,C W,0
Dup S,A A,S A,S W,0
Swap S,B S,A B,S A,S W,0
Zero 0,L L,S W,0
One 1,L L,S W,0
+ S,A S,B Add,S W,0
- S,A S,B Sub,S W,0
* S,A S,B Mult,S W,0
/ S,A S,B Div,S W,0
Not S,A -1,L L,B Mult,A 1,L L,B Add,S W,0
Negate S,A -1,L L,B Mult,S W,0
Rot S,C S,B S,A C,S A,S B,S W,0
Mod S,A A,C S,B B,S A,S /,W B,S *,W C,S -,W W,0
Continue A,A W,0
Branch Dup,W Not,W Rot,W *,W Rot,W *,W +,W S,A A,W W,0
Square Dup,W *,W W,0
Cube Dup,W Square,W *,W W,0
Fourth Square,W Square,W A,A W,0
Double Dup,W +,W W,0
-- S,A 1,L L,B Sub,S W,0
++ S,A 1,L L,B Add,S W,0
== S,A S,B Equal,S W,0
> S,A S,B Greater,S W,0
< S,A S,B Lesser,S W,0
Word1 Double,W W,0
Loop S,Z S,Y IP,X Z,W Dup,W Y,S >,W Continue,S X,S Rot,W Rot,W Branch,W W,0
Apple W,0
Orange W,0
! S,A S,P W,0
@ S,A P,S W,0
Halve S,A 2,L L,B Div,S W,0
SL 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0   
WL 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0
PS 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0 0

Overwriting code.ai


In [ ]:
N = 1024

def load(filename):
    words = np.zeros(N, dtype='object')
    codes = np.zeros(N, dtype='object')
    i = 0
    with open(filename, 'r') as f:
        for line in f:
            line = line.strip().replace(',', ' ').split()
            words[i] = line[0]
            c = line[1:]
            l = len(c)
            if l > 1:
                codes[i] = i+1
                i+=1
            codes[i:i+l] = c
            i += 1
            if l > 1:
                i+= l-1
    return words,codes

In [ ]:
def D(word):  #Dictionary
    if word in words:
        return np.where(word == words)[0][0]
    else:
        return -1

In [ ]:
def recode(codes): 
    for i in range(len(codes)):
        if codes[i] != 0 and D(codes[i]) != -1:
            codes[i] = D(codes[i])   
    return codes

In [ ]:
def setup(program):

    program = compile_program(program)

    M = np.zeros(N, dtype=int)
    M[:len(codes)] = codes
    M[D("PS"):D("PS") + len(program)] = program
    M[D("IP")] = D("PS")
    M[D("SL")] = 0 
    M[D("WL")] = 0 
    
    return M

In [ ]:
def compile_program(X):
    program = []
    for x in X.split(' '):
        
        if x.lstrip("-").isdigit():
            program += [int(x), D('L'), D('L'), D('S')]
        elif "," in x:
            a,b = recode(x.split(','))
            program += [int(a),int(b)]
        else:
            program += [D(x), D('W')]

    program += [0, D('L'), D('L'), D('IP')]
    return program

In [ ]:
def run(M):

    IP,A,B,C,Add,Sub,Mult,Div,L,P,S,SL,W,WL,Equal,Greater,Lesser = (D(x) for x in ["IP","A","B","C","Add","Sub","Mult","Div","L","P","S", "SL", "W", "WL","Equal","Greater","Lesser"])# 0,1,2,3,4,5,6,7,9,10,231,11,257  #

    while M[IP] > 0:

        M[IP] += 2

        a,b = M[IP]-2 , M[IP]-1
        i,j = M[a]    , M[b]     #copy from i paste in j
        
        if i == S and j == L:
            M[L] = i
        elif i == W and j == L:
            M[L] = i

        elif i == S:
            M[j] = M[SL + M[SL]]
            M[SL] -= 1
        elif j == S:
            M[SL] += 1
            M[SL + M[SL]] = M[i]

        elif i == W:
            M[j] = M[WL + M[WL]]
            M[WL] -= 1
        elif j == W:
            M[WL] += 1
            M[WL + M[WL]] = M[0]
            M[0] = M[i]

        elif i == P:
            M[j] = M[M[A]] 
        elif j == P:
            M[M[A]] = M[i]

        elif j == L:
            M[L] = i 

        else:
            M[j] = M[i]

        M[Add]  = M[A] + M[B]
        M[Sub]  = M[A] - M[B]
        M[Mult] = M[A] * M[B]
        M[Div]  = 0 if M[B] == 0 else M[A] // M[B]

        M[Equal]   = M[A] == M[B] 
        M[Greater] = M[A] > M[B]
        M[Lesser]  = M[A] < M[B]   

    return M

In [ ]:
words,codes = load('code.ai')
codes = recode(codes).astype(int)

In [ ]:
count = 0
for element in zip(words.tolist(),codes.tolist()):
    print(element)
    count += 1
    if count == 10:
        break

('IP', 0)
('A', 0)
('B', 0)
('C', 0)
('X', 0)
('Y', 0)
('Z', 0)
('Add', 0)
('Sub', 0)
('Mult', 0)


In [ ]:
# Test data
test_programs = [
    ("10 11 +", [21]),
    ("3 10 -", [7]),
    ("8 9 *", [72]),
    ("2 10 /", [5]),
    ("10 3 >", [0]),
    ("3 5 >", [1]),
    ("1 Not", [0]),
    ("3 10 Mod", [1]),
    ("5 Fourth", [625]),
    ("8 9 * 7 + Fourth", [38950081]),
    ("0 Not", [1]),
    ("-5 Negate", [5]),
    ("4 7 Mod", [3]),
    ("6 --", [5]),
    ("3 Double", [6]),
    ("3 5 -", [2]),
    ("3 5 >", [1]),
    ("0 5 >", [1]),
    ("6 5 >", [0]),
    ("5 Double,S Fourth,S One Branch", [625]),
    ("5 Double,S Fourth,S Zero Branch", [10]),
    ("5 Continue,S Fourth,S One Branch 22 1 +", [625,23]),
    ("1 10 Word1,S Loop",[16]),
    ("5 Apple !,W",[]),
    ("5,L L,S Apple,S !,W Apple,S @,W",[5]),
    ("4 Halve", [2]),
    ]

for i, (program, expected_answer) in enumerate(test_programs):
    M = setup(program)
    M = run(M)
    answer = M[D('SL')+1:D('SL')+1+M[D('SL')]]
    assert np.array_equal(answer, np.array(expected_answer)), f'Test {program} failed. Expected {expected_answer}, but got {answer.tolist()}.'
print('All tests passed.')

All tests passed.


In [ ]:
def REPL(M):

    while True:

        program = input()

        if program == "Exit": break
    
        program = compile_program(program)

        M[D("PS"):D("PS") + len(program)] = program
        M[0] = D("PS")
        
        M = run(M)

        # print(M[D('SL'):D('SL')+8])
        # print(M[D('SL') + M[D('SL')]])
        print(M[D('SL')+1:D('SL')+1+M[D('SL')]])

In [ ]:
# 8 9 * 7 + Fourth

In [ ]:
program = ""
M = setup(program)
REPL(M)

In [ ]:
# Fetch @
# Expects an address on stack (adddress -- value)

# Store !
# Expects a value and an address on stack (value adddress -- )

# Programs

In [ ]:
program = "Apple,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  1 321   0   0   0   0   0   0   0   0]
321


In [ ]:
program = "5,L L,A A,Apple Apple,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 5 0 0 0 0 0 0 0 0]
5


In [ ]:
program = "Apple"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[0 0 0 0 0 0 0 0 0 0]
0


In [ ]:
program = "250,L L,A 5,L L,P P,B B,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 5 0 0 0 0 0 0 0 0]
5


In [ ]:
program = "Apple,A 5,L L,P P,B B,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 5 0 0 0 0 0 0 0 0]
5


In [ ]:
program = "5,L L,S Apple,S S,A S,P P,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])
# print(M[D('Apple')])

[  1   5 321   0   0   0   0   0   0   0]
5


In [ ]:
program = "5,L L,S Apple,S !,W Apple,S S,A P,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  1   5 321   0   0   0   0   0   0   0]
5


In [ ]:
# Store !
# Expects a value and an address on stack (value adddress -- )
program = "5,L L,S Apple,S !,W"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  0   5 321   0   0   0   0   0   0   0]
0


In [ ]:
# Fetch @
# Expects an address on stack (adddress -- value)
program = "5,L L,S Apple,S !,W Apple,S @,W"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  1   5 321   0   0   0   0   0   0   0]
5


In [ ]:
program = "8 9 * 7 + Fourth"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

38950081


In [ ]:
program = "5 Fourth"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

625


In [ ]:
program = "1 2 3 Rot"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])
print(M[D('SL'):D('SL')+10])

2
[3 3 1 2 0 0 0 0 0 0]


In [ ]:
program = "1 Not"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

0


In [ ]:
program = "0 Not"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

1


In [ ]:
program = "-5 Negate"

M = setup(program)
M = run(M)
print(M[D('SL') + M[D('SL')]])

5


In [ ]:
program = "10 11 +"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])

[ 1 21 11  0  0  0  0  0  0  0]


In [ ]:
program = "10 3 Swap"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])

[ 2  3 10  0  0  0  0  0  0  0]


In [ ]:
# Word_F Word_T b Branch
# Branch expects two word address and a bool on the stack

In [ ]:
program = "5 Double,S Fourth,S One Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  2 625  23   1   0   1   0   0   0   0]
23


In [ ]:
program = "5 Double,S Fourth,S Zero Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[ 2 10 23  1  1  0  0  0  0  0]
23


In [ ]:
program = "5 Double,S Fourth,S Zero Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

In [ ]:
program = "5 Continue,S Fourth,S One Branch 22 1 +"

M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  2 625  23   1   0   1   0   0   0   0]
23


In [ ]:
program = "3 10 Mod"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[ 1  1 10  0  0  0  0  0  0  0]
1


In [ ]:
program = "4 7 Mod"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 3 7 0 0 0 0 0 0 0]
3


In [ ]:
program = "6 --"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 5 0 0 0 0 0 0 0 0]
5


In [ ]:
program = "3 Double"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 6 3 0 0 0 0 0 0 0]
6


In [ ]:
program = "3 5 -"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 2 5 0 0 0 0 0 0 0]
2


In [ ]:
program = "3 5 >"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 1 5 0 0 0 0 0 0 0]
1


In [ ]:
program = "0 5 >"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 1 5 0 0 0 0 0 0 0]
1


In [ ]:
program = "6 5 >"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 0 5 0 0 0 0 0 0 0]
0


In [ ]:
program = "1 2 3 Rot Rot"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[3 2 3 1 0 0 0 0 0 0]
1


In [ ]:
program = "1 10 Word1,S Loop"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[  1  16 182 182   1   0   0   0   0   0]
16


In [ ]:
program = "250,L L,A 5,L L,P P,B B,S"
M = setup(program)
M = run(M)
print(M[D('SL'):D('SL')+10])
print(M[D('SL') + M[D('SL')]])

[1 5 0 0 0 0 0 0 0 0]
5


In [ ]:
print(M[D('SL'):D('SL')+M[D('SL')]])

[2 5]


# Notes

We are looking at a One Instruction Set Computer OISC. The single native instruction in the 'run' virtual machine is copy,paste. The rest of the operations are memory mapped. L is for literals and allows one to convert from an address to an actual value. There are two stacks, one for parameters S at location SL and a second stack for return address called W at location WL. The language in code.ai is a forth like stack language, like forth each word is a subroutine that is called with the machine code # W where # is a place holder for the word that is looked up in the dictionary D to get location in memory where the subroutine machine code is located. The compiler build a threaded interpreted language style program that is loaded on the memory tape M and then run. This is an example of subroutine threading. Copying to W is a call (JSR) and copying from W to 0 is a return from subroutine (RTS).